<a href="https://colab.research.google.com/github/amanda-coj/MD_Suicide_Rate/blob/main/MD_Suicide_Rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Análise de Dados de Taxas de Suicídio (1985-2016)**

Este notebook tem como objetivo baixar e analisar um conjunto de dados sobre taxas de suicídio entre **1985 e 2016**. Utilizaremos ferramentas como **Python, Pandas, Seaborn e Matplotlib** para explorar os dados e visualizar tendências ao longo do tempo.


# **Con\figuração da API do Kaggle para instalar o dataset**


In [2]:
from google.colab import files

In [4]:
files.upload()



Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"amandacoj","key":"89af22b4406b8807f02c749975a286ca"}'}

In [ ]:
pip install kaggle q

In [ ]:
!mkdir ~p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list

ref                                                     title                                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
asinow/car-price-dataset                                Car Price Dataset                                 135KB  2025-01-26 19:53:28          17528        280  1.0              
ankushpanday2/heart-attack-prediction-in-united-states  Heart Attack Prediction in United States           12MB  2025-02-20 16:14:03            891         27  1.0              
mahmoudelhemaly/students-grading-dataset                Student Performance & Behavior Dataset            508KB  2025-02-17 17:38:46           2007         37  1.0              
vinothkannaece/sales-dataset                            sales dataset                                      27K

In [ ]:
!kaggle datasets download -d russellyates88/suicide-rates-overview-1985-to-2016

Dataset URL: https://www.kaggle.com/datasets/russellyates88/suicide-rates-overview-1985-to-2016
License(s): world-bank
  0% 0.00/397k [00:00<?, ?B/s]
100% 397k/397k [00:00<00:00, 65.8MB/s]


In [ ]:
!unzip suicide-rates-overview-1985-to-2016.zip

Archive:  suicide-rates-overview-1985-to-2016.zip
  inflating: master.csv              


In [ ]:
!pip install basemap
!pip install basemap-data-hires


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 942.4/942.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 55.3 MB/s eta 0:00:00
  Attempting uninstall: pyproj
    Found existing installation: pyproj 3.7.1
    Uninstalling pyproj-3.7.1:
      Successfully uninstalled pyproj-3.7.1
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.10.0
    Uninstalling matplotlib-3.10.0:
      Successfully uninstalled matplotlib-3.10.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 MB 6.1 MB/s eta 0:00:00


In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.basemap import Basemap
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


import warnings
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

ModuleNotFoundError: No module named 'mpl_toolkits.basemap'

In [ ]:
df = pd.read_csv('suicide-rates-overview-1985-to-2016.zip')

In [ ]:
df.columns=["country","year","gender","age_group","suicide_number","population","sui_pop","country_year","hdi","gdp_for_year","gdp_per_capita","generation"]


In [ ]:
df.columns=["country","year","gender","age_group","suicide_number","population","sui_pop","country_year","hdi","gdp_for_year","gdp_per_capita","generation"]

In [ ]:
df.drop(["country_year","gdp_for_year","gdp_per_capita","hdi"],axis=1,inplace=True)
df.head()

,country,year,gender,age_group,suicide_number,population,sui_pop,generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Boomers


In [ ]:
df.age_group.unique()


array(['15-24 years', '35-54 years', '75+ years', '25-34 years',
       '55-74 years', '5-14 years'], dtype=object)

In [ ]:
df["age_group"]=df["age_group"].apply(lambda x: str(x).replace('5-14 years','child') if '5-14 years' in str(x) else str(x))
df["age_group"]=df["age_group"].apply(lambda x: str(x).replace('15-24 years','youth') if '15-24 years' in str(x) else str(x))
df["age_group"]=df["age_group"].apply(lambda x: str(x).replace('25-34 years','young adult') if '25-34 years' in str(x) else str(x))
df["age_group"]=df["age_group"].apply(lambda x: str(x).replace('35-54 years','early adult') if '35-54 years' in str(x) else str(x))
df["age_group"]=df["age_group"].apply(lambda x: str(x).replace('55-74 years','adult') if '55-74 years' in str(x) else str(x))
df["age_group"]=df["age_group"].apply(lambda x: str(x).replace('75+ years','senior') if '75+ years' in str(x) else str(x))